# Aprendizaje de Máquina con Big Data y Apache Spark
## Clasificación
![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)

Para el procesamiento de Big Data utilizando Ciencia de Datos/Analítica de Datos/Aprendizaje de Máquina, existe una metodología ampliamente utilizada en la industria conocida como CRISP-DM creada por IBM, el siguiente diagrama representa la secuencia de pasos correspondientes a esta metodología:

![CRISP-DM](https://www.ibm.com/docs/es/SS3RA7_sub/modeler_crispdm_ddita/clementine/images/crisp_process.jpg)

En este diagrama se identifican las siguientes etapas:

1. Entendimiento del negocio (objetivos)
2. Exploración de los datos
3. Preparación de los datos
4. Modelado
5. Evaluación
6. Despliegue

El cual representa un proceso iterativo, que comienza con el entendimiento del negocio y termina, y vuelve a comenzar, con la evaluación de resultados.

## Descripción del problema

Con sus conocimientos en ciencia de datos, se le ha pedido que a partir de un [conjunto de datos](https://archive.ics.uci.edu/ml/datasets/iris) con medidas de los pétalos y sépalos de distintas clases de la flor iris, entrene un modelo para identificar automáticamente a que especie pertenece una flor de acuerdo a sus medidas.

### Configuración del ambiente de Google Colaboratory

In [ ]:
# Download Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Next, we will install Apache Spark 3.0.1 with Hadoop 2.7 from here.
!wget https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
# Now, we just need to unzip that folder.
!tar xf spark-3.3.2-bin-hadoop3.tgz

# Setting JVM and Spark path variables
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

# Installing required packages
!pip install pyspark==3.3.2
!pip install findspark

In [ ]:
import datetime as dt
import findspark
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyspark.ml as ml
from pyspark.sql import functions as fct
from pyspark.sql import SparkSession
from pyspark.sql.types import *

findspark.init()

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
iris_data = iris["data"]
iris_target = np.array([iris["target_names"][int(idx)] for idx in iris["target"]])

### Crear Sesión de Spark e importar los datos

In [ ]:
ss = (SparkSession
      .builder
      .appName("data_exploration_preparation")
      .getOrCreate())

In [ ]:
lst = iris_data.tolist()
lst[:][4]=iris_target.tolist()

In [ ]:
schema = StructType([StructField(iris["feature_names"][0], DoubleType(), True),
                     StructField(iris["feature_names"][1], DoubleType(), True),
                     StructField(iris["feature_names"][2], DoubleType(), True),
                     StructField(iris["feature_names"][3], DoubleType(), True),
                     StructField("class", StringType(), True)])
iris_data_ss = ss.createDataFrame(data = [x+[y] for x,y in zip(iris_data.tolist(),iris_target.tolist())], schema = schema)

### División en el conjunto de entrenamiento y conjunto de evaluación

In [ ]:
train_size = 0.7 # Tamaño del conjunto de entrenamiento: 70%
test_size = 0.3 # Tamaño del conjunto de evaluación: 30%
iris_data_train, iris_data_test = iris_data_ss.randomSplit([train_size, test_size], seed=42)
iris_data_pd = iris_data_train.toPandas() # Convertirlo a un DataFrame de pandas para generar visualizaciones

### Exploración de los datos

In [ ]:
iris_data_train.printSchema() # Esquema relacional del conjunto de datos

In [ ]:
iris_data_train.show(10) # Primeros 10 registros del conjunto de datos

In [ ]:
(iris_data_train
 .describe() # Características estadísticas básicas
 .show())

In [ ]:
iris_data_pd.hist(bins=50, figsize=(12, 8)) # Calcular y graficar el histograma de cada característica
plt.show()

#### Correlaciones
La correlación es una medida estadística que expresa hasta qué punto dos variables están relacionadas linealmente (esto es, cambian conjuntamente a una tasa constante). Es una herramienta común para describir relaciones simples sin hacer afirmaciones sobre causa y efecto [[ref](https://www.jmp.com/es_co/statistics-knowledge-portal/what-is-correlation.html)].

In [ ]:
pd.plotting.scatter_matrix(iris_data_pd[iris["feature_names"]], figsize=(12, 8))
plt.show()

In [ ]:
color_map = {cls: color for cls, color in zip(iris["target_names"], ["red", "green", "blue"])}
print(color_map)
pd.plotting.scatter_matrix(iris_data_pd[iris["feature_names"]], color=[color_map[cls] for cls in iris_data_pd["class"].values], figsize=(12, 8))
plt.show()

### Preparación de los datos

In [ ]:
stringIndexer = ml.feature.StringIndexer(inputCol="class", outputCol="ordinal_class", stringOrderType="frequencyDesc")

onehotencoder = ml.feature.OneHotEncoder(inputCol="ordinal_class", outputCol="onehot_class")

columns_to_scale = iris["feature_names"]

assemblers = [ml.feature.VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_scale]

scalers = [ml.feature.MinMaxScaler(inputCol=col + "_vec", outputCol="scaled_" + col) for col in columns_to_scale]

feature_assembler = ml.feature.VectorAssembler(inputCols=["scaled_" + col for col in columns_to_scale], outputCol="features")

sqlTrans = ml.feature.SQLTransformer(statement="SELECT features, ordinal_class AS label FROM __THIS__")

preprocess_pipeline = ml.Pipeline(stages=[stringIndexer, onehotencoder]+assemblers+scalers+[feature_assembler, sqlTrans])

In [ ]:
pipeline_model = preprocess_pipeline.fit(iris_data_train)
pipeline_model.transform(iris_data_train).show()

### Entrenamiento y evaluación de modelos

#### Regresión logística

In [ ]:
Logistic_regression = ml.classification.LogisticRegression()
model_pipeline = ml.Pipeline(stages=[preprocess_pipeline, Logistic_regression])

model = model_pipeline.fit(iris_data_train)

train_predictions = model.transform(iris_data_train)
train_predictions.show()

In [ ]:
dir(model.stages[-1].summary)

In [ ]:
print(f"Precisión: {model.stages[-1].summary.accuracy}")
print(f"F-Score: {model.stages[-1].summary.fMeasureByLabel()}")

In [ ]:
evaluator = ml.evaluation.MulticlassClassificationEvaluator(metricName = "accuracy")
print("Accuracy on train data = %g" % evaluator.evaluate(train_predictions))

In [ ]:
predictions = model.transform(iris_data_test)
predictions.show(5)
print("Accuracy on test data = %g" % evaluator.evaluate(predictions))

#### Árboles de decisión

In [ ]:
decisionTree = ml.classification.DecisionTreeClassifier()
model_pipeline = ml.Pipeline(stages=[preprocess_pipeline, decisionTree])

model = model_pipeline.fit(iris_data_train)

train_predictions = model.transform(iris_data_train)
train_predictions.show()

In [ ]:
print("Accuracy on train data = %g" % evaluator.evaluate(train_predictions))

In [ ]:
predictions = model.transform(iris_data_test)
predictions.show(5)
print("Accuracy on test data = %g" % evaluator.evaluate(predictions))

#### Perceptron multicapa o Red Neuronal

In [ ]:
layers = [4, 10, 10, 3]

trainer = ml.classification.MultilayerPerceptronClassifier(maxIter=500, layers=layers, blockSize=128, seed=1234)
model_pipeline = ml.Pipeline(stages=[preprocess_pipeline, trainer])

model = model_pipeline.fit(iris_data_train)

train_predictions = model.transform(iris_data_train)
train_predictions.show()

In [ ]:
print("Accuracy on train data = %g" % evaluator.evaluate(train_predictions))

In [ ]:
predictions = model.transform(iris_data_test)
predictions.show(5)
print("Accuracy on test data = %g" % evaluator.evaluate(predictions))